In [0]:
pip install tweepy

Python interpreter will be restarted.
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Not uninstalling requests at /databricks/python3/lib/python3.8/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-0a9d0f7e-6d0c-476b-ab8d-ed7749b3d909
    Can't uninstall 'requests'. No files were found to uninstall.
Python interpreter will be restarted.


In [0]:
from pyspark.sql import SparkSession
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
import socket
import json
import pandas as pd

In [0]:
consumer_key = 'ctHXYY0FWix0wjwHjIYW5CybR'
consumer_secret = 's5BLJEG9sTcAS6tIbZTIUrx2fOH9aOg5if5N6LtV3Q3tExfoV6'
access_token = '2432314723-9QxUMUy7rZhfrsIjpP8246dVXZUuuGqcDa7SyAb'
access_token_secret = 'N4ufqqPMwVVxxnIxDjzoAKSM4M6ihMmCAu5opJE1OUwXW'

In [0]:
class TweetsListener(Stream):
  # tweet object listens for the tweets
  def __init__(self, *args, csocket):
        super().__init__(*args)
        self.client_socket = csocket
  def on_data(self, data):
    try:  
      msg = json.loads( data )
      print("new message")
      # if tweet is longer than 140 characters
      if "extended_tweet" in msg:
        # add at the end of each tweet "t_end" 
        self.client_socket\
            .send(str(msg['extended_tweet']['full_text']+"t_end")\
            .encode('utf-8'))         
        print(msg['extended_tweet']['full_text'])
      else:
        # add at the end of each tweet "t_end" 
        self.client_socket\
            .send(str(msg['text']+"t_end")\
            .encode('utf-8'))
        print(msg['text'])
      return True
    except BaseException as e:
        print("Error on_data: %s" % str(e))
    return True
  def on_error(self, status):
    print(status)
    return True

In [0]:
def send_data(c_socket):
    twtr_stream = TweetsListener(
        consumer_key, consumer_secret,
        access_token, access_token_secret,
        csocket=c_socket
    )
    twtr_stream.filter(languages=["en"], track=['Fifa'])

In [0]:
if __name__ == "__main__":
    # server (local machine) creates listening socket
    s = socket.socket()
    host = "0.0.0.0"    
    port = 5555
    s.bind((host, port))
    print('socket is ready')
    # server (local machine) listens for connections
    s.listen(4)
    print('socket is listening')
    # return the socket and the address on the other side of the connection (client side)
    c_socket, addr = s.accept()
    print("Received request from: " + str(addr))
    # select here the keyword for the tweet data
    # sendData(c_socket, keyword = ['piano'])
    send_data(c_socket)

socket is ready
socket is listening
Received request from: ('127.0.0.1', 60122)
new message
RT @plstreaminglive: LIONEL MESSI FINALLY GETS HIS DREAM THE FIFA WORLD CUP! 🐐🏆

NOW HE HAS WON ALL TROPHIES IN HIS CAREER 

A MOMENT EVERY…
new message
@RKuhT5CLSW2SPZH @sportbible FIFA planted a virus in the french hotel.
new message
By the next FIFA world cup, I want to be rich enough to watch it in the stadium. me and @PratulK11🫡 
#FIFAWorldCup
new message
@ShadayaKnight Mbappe is a World Cup winner. It was going to be his second if it wasn't the corrupt match officials and FIFA https://t.co/lasJDQECLo
new message
RT @AlbicelesteTalk: OFFICIAL: Lionel Messi is the FIFA World Cup Player of the Tournament. 🐐🏆
new message
RT @MelnykAndrij: A must-see appeal of President @ZelenskyyUa on the occasion of football World Cup 2022 final. 
PS: rejected by #FIFA 🤷‍♂️…
new message
RT @FOXSoccer: This time lapse of Buenos Aires after winning the FIFA World Cup is WILD 🤯🇦🇷 https://t.co/F6ixRsKvMK
new mess